In [ ]:
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from src.consts import *
from src.validation import validate_jumpstart_cube, display_validate_results
from src.coherence import analyze_deck_theme_coherence_enhanced
from src.improve import apply_swap, repair_cube_deck_sizes

load_dotenv(override=True)
openai = OpenAI()
anthropic = Anthropic() 

# Load the data files
oracle_df = pd.read_csv('ThePauperCube_oracle_with_pt.csv')
# cube_df = pd.read_csv('JumpstartCube_ThePauperCube_ULTIMATE_Final_v2.csv')

# Optimise

In [ ]:
# You can also analyze a specific deck by name
# Example: Analyze the "Green Big Creatures" deck

import importlib
import src.process
import src.improve
importlib.reload(src.process)
importlib.reload(src.improve)

from src.process import optimize_deck_coherence, clear_swap_history

# Clear any previous swap history to prevent issues with oscillation
clear_swap_history()

coherence = analyze_deck_theme_coherence_enhanced(cube_df, oracle_df)
total_coherence = 0

for i in range(1, 2):
    for deck_name in coherence.keys():
        cube_df = optimize_deck_coherence(cube_df=cube_df, oracle_df=oracle_df, deck_name=deck_name)

In [ ]:
# You can also analyze a specific deck by name
# Example: Analyze the "Green Big Creatures" deck

import importlib
import src.process
import src.improve
importlib.reload(src.process)
importlib.reload(src.improve)

from src.process import optimize_deck_coherence, clear_swap_history

# Clear any previous swap history to prevent issues with oscillation
clear_swap_history()

coherence = analyze_deck_theme_coherence_enhanced(cube_df, oracle_df)
total_coherence = sum(result['overall_coherence'] for result in coherence.values())

improvement = True
iteration = 0
max_iterations = 20  # Prevent infinite loops

while improvement and iteration < max_iterations:
    prev_total_coherence = total_coherence
    cube_df = optimize_deck_coherence(cube_df=cube_df, oracle_df=oracle_df)
    coherence = analyze_deck_theme_coherence_enhanced(cube_df, oracle_df)
    total_coherence = sum(result['overall_coherence'] for result in coherence.values())
    improvement = total_coherence > prev_total_coherence
    iteration += 1
    print(f"Iteration {iteration}: Total coherence: {total_coherence:.2f} (improvement: {total_coherence - prev_total_coherence:.2f})")
    
    if not improvement:
        print("No further improvements found - optimization complete!")
    elif iteration >= max_iterations:
        print("Maximum iterations reached - stopping optimization")

# Save to file

In [ ]:
# from src.export import export_cube_to_csv

# export_cube_to_csv(cube_df, oracle_df, 'JumpstartCube_ThePauperCube_ULTIMATE_Final_v2.csv')

# Analysis

In [ ]:
display_validate_results(validate_jumpstart_cube(cube_df, oracle_df))